<a href="https://colab.research.google.com/github/aishadvitya/llm_playground/blob/main/multistep_decomposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/query_transformations/SimpleIndexDemo-multistep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Step Query Engine

We have a multi-step query engine that's able to decompose a complex query into sequential subquestions. This
guide walks you through how to set it up!

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [1]:
%pip install llama-index-llms-openai

In [2]:
!pip install llama-index

#### Download Data

#### Load documents, build the VectorStoreIndex

In [3]:
import os

os.environ["OPENAI_API_KEY"] = "sk..."

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

In [5]:
# LLM (gpt-3.5)
gpt35 = OpenAI(temperature=0, model="gpt-3.5-turbo")



In [6]:
# load documents
documents = SimpleDirectoryReader(input_files=["/content/call_transcript_elv.pdf"]).load_data()

In [7]:
index = VectorStoreIndex.from_documents(documents)

#### Query Index

In [8]:
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)
# gpt-3
step_decompose_transform_gpt3 = StepDecomposeQueryTransform(
    llm=gpt35, verbose=True
)

In [9]:
index_summary = "Used to answer questions about the document"

In [11]:
# set Logging to DEBUG for more detailed outputs
from llama_index.core.query_engine import MultiStepQueryEngine

query_engine = index.as_query_engine(llm=gpt35)
query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform_gpt3,
    index_summary=index_summary,
)
response_gpt35 = query_engine.query(
    "what was gaels overview of the financial earnings",
)

> Current query: what was gaels overview of the financial earnings
> New query: What is Gael's profession or occupation?
> Current query: what was gaels overview of the financial earnings
> New query: What specific financial information did Gael provide in the overview of the earnings?
> Current query: what was gaels overview of the financial earnings
> New query: What was the percentage growth in Elevance Health's earnings per share according to Gael's overview?


In [12]:
display(Markdown(f"<b>{response_gpt435</b>"))

<b>Gail provided an overview of Elevance Health's first quarter GAAP earnings per share of $9.59 and adjusted diluted earnings per share of $10.64, indicating a growth of 12.5%. She also mentioned that the company raised its guidance for adjusted earnings per share by $0.10 to be greater than $37.20.</b>

In [13]:
sub_qa = response_gpt35.metadata["sub_qa"]
tuples = [(t[0], t[1].response) for t in sub_qa]
print(tuples)

[("What is Gael's profession or occupation?", 'Gail Boudreau is a moderator or host for the conference call.'), ('What specific financial information did Gael provide in the overview of the earnings?', "Gail provided information on Elevance Health's first quarter GAAP earnings per share of $9.59 and adjusted diluted earnings per share of $10.64, showing a growth of 12.5%. Additionally, she mentioned that the company increased its guidance for adjusted earnings per share by $0.10 to be greater than $37.20."), ("What was the percentage growth in Elevance Health's earnings per share according to Gael's overview?", "The percentage growth in Elevance Health's earnings per share according to Gail's overview was 12.5%.")]
